In [1]:
import os
import pandas as pd
import json
import numpy as np


In [2]:
DATA_DIR = '02_zephyr-internal-CoT'
INPUT_DATA_PATH = f'{DATA_DIR}/inference_data.jsonl'
EXTRACTED_TABLE_PATH = f'{DATA_DIR}/infer_table.csv'
LABEL_KEY = "zephyr"

# Extract

## Read data

In [3]:
def read_inference(address):
    json_list = []
    with open(address, 'r') as file:
        for line in file:
            data = json.loads(line)
            json_list.append(data)
    return json_list

In [4]:
inference_data =  read_inference(INPUT_DATA_PATH)

print(f"Number of loaded data: {len(inference_data)}")

Number of loaded data: 120


## Extract option

In [5]:
def find_last_mentioned_option(text: str):
    """return option index as [0, 1, 2, 3]"""
    option_key = ["Option 1", "Option 2", "Option 3", "Option 4"]
    last_mentioned = [text.rfind(opt) for opt in option_key]
    return (np.argmax(last_mentioned))

In [6]:
data = []

for ds in inference_data:
    option_index = find_last_mentioned_option(ds[LABEL_KEY])
    data.append({
        "question": ds['question'],
        "0": ds['option 1'],
        "1": ds['option 2'],
        "2": ds['option 3'],
        "3": ds['option 4'],
        LABEL_KEY: option_index,
        LABEL_KEY+"-raw": ds[LABEL_KEY],
    })
    
df = pd.DataFrame(data)
df.to_csv(EXTRACTED_TABLE_PATH, index=False)

# Prepare Submission

In [7]:
submission_file = os.path.join(DATA_DIR, "answer_sen.txt")

In [8]:
data = pd.read_csv(EXTRACTED_TABLE_PATH)

In [9]:
submission = [str(ds[LABEL_KEY]) for idx, ds in data.iterrows()]

In [10]:
submission_content = '\n'.join(submission)

In [11]:
with open(submission_file, 'w') as f:
    f.write(submission_content)
    
with open("answer_sen.txt", 'w') as f:
    f.write(submission_content)

In [12]:
import zipfile

zip_submission_path = submission_file[:submission_file.rfind('.')] + ".zip"
zip_submission_path = zip_submission_path.replace("answer_sen", "submission")

In [13]:
zipfile.ZipFile(zip_submission_path, mode='w').write("answer_sen.txt")